In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, helper, tx

### This is a Jupyter Notebook
You can write Python code and it will execute. You can write the typical 'hello world' program like this:

```python
print('hello world')
```

You can execute by pressing shift-enter. Try it! You can also click the Run button in the toolbar.


In [2]:
print('hello world')

hello world


### Imports

You already have unit tests that are written for you.
Your task is to make them pass.
We can import various modules to make our experience using Jupyter more pleasant.
This way, making everything work will be a lot easier.


In [3]:
# this is how you import an entire module
import helper
# this is how you import a particular function, class or constant
from helper import little_endian_to_int
# used in the next exercise
some_long_variable_name = 'something'

### Exercise 1
#### Jupyter Tips

The two most useful commands are tab and shift-tab

Tab lets you tab-complete. Try pressing tab after the `some` below. This will complete to the variable name that's there from the last cell.

Shift-Tab gives you a function/method signature. Try pressing shift-tab after the `little_endian_to_int` below. That's also there from the last cell.


In [4]:
# Exercise 1

some_long_variable_name
little_endian_to_int(b'\x00')

0

### Exercise 2


This is a refresher on how some of the byte encoding and reading works. In particular, this exercise is to give you a refresher on how streams work.

Open [helper.py](/edit/session0/helper.py) and implement the `read_varstr` and `encode_varstr` functions. Once you're done editing, run the cell below.


#### Make [this test](/edit/session0/helper.py) pass: `helper.py:HelperTest:test_varstr`

In [5]:
# Exercise 2

reload(helper)
run(helper.HelperTest('test_varstr'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 3
#### ECDSA Refresher

Sign the message `b'I completed the Programming Blockchain Seminar and/or the Programming Bitcoin book'` with a private key of your choosing. We'll use this key for at least a part of this class, so please keep it handy and back it up!


In [6]:
# Exercise 3

from ecc import PrivateKey
from helper import hash256, little_endian_to_int
# pick a secret passphrase. Something like your email address and your name should be fine.
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
# this is the message we'll be signing
message = b'I completed the Programming Blockchain Seminar and/or the Programming Bitcoin book'
# we're converting the passphrase into a secret number for use in the private key
secret = little_endian_to_int(hash256(passphrase))
# create the private key object
private_key = PrivateKey(secret)
# create the z by converting the hash256 of the message to a big endian integer using int.from_bytes(x, 'big')
z = int.from_bytes(hash256(message), 'big')
# sign the message with the private key
signature = private_key.sign(z)
# print the hex of the der version of the signature
print(signature.der().hex())
# verify the signature using the public key (private_key.point)
print(private_key.point.verify(z, signature))

30450221009daf110ed47cf2e5b0b59e766af5476a35dcaadfbbd03250857cd7c2d2dc661302206cc4b3a5347e9215915f9e31084d4f49e07c34717a79d01aec259fd65381e74e
True


### Exercise 4


It's annoying to have to calculate z every time we want to sign or verify a message. Write the `sign_message` method in `PrivateKey` and the `verify_message` method in `S256Point` to make signing/verifying messages easy.


#### Make [this test](/edit/session0/ecc.py) pass: `ecc.py:PrivateKeyTest:test_sign_message`

In [7]:
# Exercise 4

reload(ecc)
run(ecc.PrivateKeyTest('test_sign_message'))

.
----------------------------------------------------------------------
Ran 1 test in 0.241s

OK


### Exercise 5
#### Address Refresher

Get your testnet address from your private key used above.


In [8]:
# Exercise 5

# get the public key for your private key (the private_key variable should still be in scope)
public_key = private_key.point
# print the address using the address method. remember to pass in testnet=True!
print(public_key.address(testnet=True))

mmMupB3E1xNfRZktg6N9ehuAZWCkrCSXSp


### Exercise 6
#### Transaction Refresher

Send yourself some testnet coins to the address from the previous exercise [using this site](https://faucet.programmingbitcoin.com).
Then create a transaction to send all the coins to `mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv` BUT DO NOT BROADCAST IT YET!
This is a one input, one output transaction.


In [9]:
# Exercise 6

from helper import decode_base58
from script import p2pkh_script
from tx import Tx, TxIn, TxOut
# this should be the transaction ID and index from the transaction from the faucet
prev_tx_hex = 'aec4b5bfa8952a80a93e9afd437f4783e51d363303c021f68c7a614ca8a153e4'
prev_index = 1
prev_tx = bytes.fromhex(prev_tx_hex)
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
fee = 5000
# create the transaction input
tx_in = TxIn(prev_tx, prev_index)
# calculate the output amount
output_amount = tx_in.value(testnet=True) - fee
# calculate the hash160 using decode_base58 on the target_address
h160 = decode_base58(target_address)
# convert the h160 to a p2pkh script using p2pkh_script
script_pubkey = p2pkh_script(h160)
# create the transaction output
tx_out = TxOut(output_amount, script_pubkey)
# create the transaction with version=1, locktime=0, testnet=True
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the transaction's only input using sign_p2pkh
# the private key from the previous exercise should be in scope
tx_obj.sign_p2pkh(0, private_key)
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())

0100000001e453a1a84c617a8cf621c00333361de583477f43fd9a3ea9802a95a8bfb5c4ae010000006b483045022100919c5f390ac25e7aa66c250bb55f37a4ccf565a9681343cf822220c06879b4e9022027d4d2f691e31749280d30bf840160b6b8b682fff4481e84094a732b791917d2012102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5dffffffff01f7f03900000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000


### Exercise 7


It's annoying to have to already know what the previous output's script type was to sign it as the caller. Create a method to sign any input, though for now, that should only be p2pkh.


#### Make [this test](/edit/session0/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [10]:
# Exercise 7

reload(tx)
run(tx.TxTest('test_sign_input'))

.
----------------------------------------------------------------------
Ran 1 test in 0.185s

OK


### Exercise 8
#### Network Refresher

Broadcast the transaction created in the previous exercise directly on the testnet network.


In [11]:
# Exercise 8

from network import SimpleNode
from time import sleep
# connect to testnet.programmingbitcoin.com in testnet mode using SimpleNode
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
# complete the handshake
node.handshake()
# send this signed transaction on the network
node.send(tx_obj)
# wait a sec so this message goes through to the other node sleep(1)
sleep(1)
# now check to see if the tx has been accepted using SimpleNode.is_tx_accepted()
if node.is_tx_accepted(tx_obj):
    print('success!')
    print(tx_obj.id())

success!
f6afd234109e5aa6ba388d386fc94c7109bcf1f4e618aaeffdf037e709ab3a77
